In [56]:
# !pip install --quiet -U langgraph

In [57]:
import os

In [58]:
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

from langchain import hub
from langchain.agents import create_openai_functions_agent

In [59]:
search = GoogleSearchAPIWrapper()

search_tool = Tool(
    name="google_search",
    description="Search Google for recent results. This will return relevant information from the web with links to the sources.",
    func=lambda query: f"""
        Result for query: {query}
        {os.linesep.join([f"Title: {result['title']}  Peek into article {result['snippet']}  Link for more info: {result['link']}" for result in search.results(query, num_results=5)])}
    """,
)

tools=[search_tool]

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
        You are a research agent. You will be given a topic and you need to do research on it. You have access to tools that you can use to help you get information while preparing the research report.
        You need to create a detailed research report that covers the topic in detail. First gather all the information you can find on the topic on the web.
        """
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "Research {research_topic}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm = ChatOpenAI(
    model="gpt-4o",
    max_retries=2,
    temperature=0.25,
)

agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [60]:
import operator
from typing import Annotated, TypedDict, Union

from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    # The input string
    research_topic: str
    # The list of previous messages in the conversation
    chat_history: list[BaseMessage]
    # The outcome of a given call to the agent
    # Needs `None` as a valid type, since this is what this will start as
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # List of actions and corresponding observations
    # Here we annotate this with `operator.add` to indicate that operations to
    # this state should be ADDED to the existing values (not overwrite it)
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

In [61]:
from langchain_core.agents import AgentFinish

from langgraph.prebuilt.tool_executor import ToolExecutor

# This a helper class we have that is useful for running tools
# It takes in an agent action and calls that tool and returns the result
tool_executor = ToolExecutor(tools)


# Define the agent
def run_agent(data):
    print("faiz here", data)
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}


# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}


# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

In [62]:
from langgraph.graph import END, StateGraph, START

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [63]:
input = {"research_topic": "India 2024 lok sabha elections", "chat_history": [], "intermediate_steps": [], "agent_outcome": None}
app.invoke(input=input)

faiz here {'research_topic': 'India 2024 lok sabha elections', 'chat_history': [], 'agent_outcome': None, 'intermediate_steps': []}
faiz here {'research_topic': 'India 2024 lok sabha elections', 'chat_history': [], 'agent_outcome': AgentActionMessageLog(tool='google_search', tool_input='India 2024 Lok Sabha elections', log='\nInvoking: `google_search` with `India 2024 Lok Sabha elections`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"India 2024 Lok Sabha elections"}', 'name': 'google_search'}}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 148, 'total_tokens': 170}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_dd932ca5d1', 'finish_reason': 'function_call', 'logprobs': None}, id='run-32d7f75b-57e3-424c-af67-90671bcd28f3-0', usage_metadata={'input_tokens': 148, 'output_tokens': 22, 'total_tokens': 170})]), 'intermediate_steps': [(AgentActionMessageLog(tool='google_search', tool_input

{'research_topic': 'India 2024 lok sabha elections',
 'chat_history': [],
 'agent_outcome': AgentFinish(return_values={'output': "### Research Report: India 2024 Lok Sabha Elections\n\n#### Overview\nThe 2024 Indian general election, also known as the Lok Sabha elections, were held to elect members to the 18th Lok Sabha, the lower house of India's Parliament. The elections were conducted in seven phases from April 19 to June 1, 2024. The counting of votes took place shortly after the final phase, and the results were declared in early June 2024.\n\n#### Key Highlights\n1. **Election Phases**: The election was conducted in seven phases to manage the vast electorate and ensure security and logistical efficiency.\n2. **Total Seats**: The election was held for all 543 Lok Sabha seats.\n3. **Major Parties**:\n   - **Bharatiya Janata Party (BJP)**: Led by Prime Minister Narendra Modi.\n   - **Indian National Congress (INC)**: Led by Rahul Gandhi.\n   - **Other Regional Parties**: Including t

In [45]:
agent_runnable.invoke({"research_topic": "India 2024 lok sabha elections", "chat_history": [], "intermediate_steps": [], "agent_outcome": None})

AgentActionMessageLog(tool='google_search', tool_input='India 2024 Lok Sabha elections', log='\nInvoking: `google_search` with `India 2024 Lok Sabha elections`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"India 2024 Lok Sabha elections"}', 'name': 'google_search'}}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 148, 'total_tokens': 170}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_dd932ca5d1', 'finish_reason': 'function_call', 'logprobs': None}, id='run-1a50330f-1c12-423f-8fb3-dbd3abd6013a-0', usage_metadata={'input_tokens': 148, 'output_tokens': 22, 'total_tokens': 170})])